# ML Project 2
## Training/validation/testing

Requirements:

- Code/instructions for loading
saved model and running model
on 10 example cases (e.g.,
images)

- Okay to document results on
these 10 cases here (rather than
including them as part of
training/validation/testing
notebook)

### Instructions

Code should just be plug and play, though ensure the following:

- `best_model.pt` is in the root directory
- the test images are in a folder called `testImages` and the folder is in the root directory

## Imports and such

In [13]:
import subprocess

# Installation on Google Colab
try:
    import os
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch', 'torchvision'])
    subprocess.run(['mkdir', '-p', 'datasets'])
    subprocess.run(['wget', '-nc', '--no-check-certificate',
                   'https://download.pytorch.org/tutorial/hymenoptera_data.zip', '-P', 'datasets'])
    subprocess.run(
        ['unzip', '-u', 'datasets/hymenoptera_data.zip', '-d' 'datasets'])
except ImportError:
    pass

In [14]:
import os
from urllib import request
from zipfile import ZipFile

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, models, transforms

from skorch import NeuralNetClassifier
from skorch.helper import predefined_split

from skorch.callbacks import LRScheduler
from skorch.callbacks import Checkpoint
from skorch.callbacks import Freezer

torch.manual_seed(360)

## Running our best model

In [15]:
from PIL import Image

class PretrainedModel(nn.Module):
    def __init__(self, output_features):
        super().__init__()
        model = models.resnet18(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, output_features)
        self.model = model

    def forward(self, x):
        return self.model(x)

model_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

net = NeuralNetClassifier(
    PretrainedModel,
    criterion=nn.CrossEntropyLoss,
    module__output_features=5,
)

net.initialize()
net.load_params(f_params='best_model.pt')

test_dir = 'testImages'
test_images = os.listdir(test_dir)

classes = ['Baseball', 'Basketball', 'Football', 'Hockey', 'Volleyball']
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}

correct = 0
total = 0

for img_name in test_images:
    img_path = os.path.join(test_dir, img_name)
    img = Image.open(img_path)
    img_transformed = model_transforms(img).unsqueeze(0)

    # Extract class name from filename by removing digits and extension
    class_name = ''.join(filter(str.isalpha, img_name.split('.')[0])).capitalize()

    # Predict
    output = net.predict(img_transformed)
    predicted = output[0]

    # Check prediction
    if predicted == class_to_idx[class_name]:
        correct += 1
    else:
        print(f'Predicted: {classes[predicted]}, Actual: {class_name}, filename: {img_name}')
    total += 1

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

Predicted: Football, Actual: Baseball, filename: baseball4.jpg
Predicted: Basketball, Actual: Baseball, filename: baseball5.jpg
Accuracy: 92.00%
